<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [5]:
import os
os.chdir('/aiml/data/DevMind/06.SRC/01.TestCaseBot/02.TestAWBot/AW_Recommand_V2')

In [200]:
# %load /aiml/data/DevMind/06.SRC/01.TestCaseBot/02.TestAWBot/AW_Recommand_V2/aw_recommand/infer/search_aw_by_key_word.py

import logging
import pandas as pd

class Search_AW_By_Key(object):
    def __init__(self, logger=None):
        # AW 信息列表
        df = pd.read_csv('./data/sentence_embeding/clean_data.csv')
        self.df_aw = df.loc[:, ['_id', 'new_attr_aw']]

        if logger:
            self.logger = logger
        else:
            self.logger = logging.getLogger()
            self.logger.setLevel(logging.INFO)

    def update_score(self, word_weight_dict):
        def get_score_by_str(input_str):
            input_list = input_str.split()
            score_list = [word_weight_dict.get(x, 0) for x in sorted(list(set(input_list)), key=input_list.index)]
            return score_list
        score_list = self.df_aw['new_attr_aw'].apply(get_score_by_str)
        score = score_list.apply(lambda x: sum(x) + 1.0 / (pow(len(x) - len(word_weight_dict), 2) + 1))
        self.df_aw = self.df_aw.assign(score_list=score_list)
        self.df_aw = self.df_aw.assign(score=score)
        if logger.level <= logging.INFO:
            self.df_aw.to_csv('./log/aw_score.csv', index=False)
        return score_list, score


    def get_top_n_score_aw(self, word_weight_dict):
        self.update_score(word_weight_dict)
        return self.df_aw.sort_values('score', ascending=False).head(10).new_attr_aw.tolist()


In [201]:
tmp1 = Search_AW_By_Key()
tmp_dict= {'deploy': 1, 'remove': 1, 'openflow': 2}

In [202]:
tmp1.get_top_n_score_aw(tmp_dict)

['unset openflow auth',
 'get openflow session',
 'config openflow controller',
 'get openflow session',
 'get openflow statistics',
 'get openflow session',
 'config openflow agent',
 'set openflow auth',
 'get openflow table socket',
 'get openflow fpm table']

In [208]:
from gensim.models import word2vec
from gensim.models import KeyedVectors

In [227]:
path_pretrain_model_in_aw = './data/sentence_embeding/GoogleNews-vectors-negative300.model.vec'
pretrain_model_in_aw = KeyedVectors.load_word2vec_format(path_pretrain_model_in_aw)

In [233]:
pretrain_model_in_aw.similar_by_vector(pretrain_model_in_aw.get_vector('do'))

[('do', 1.0),
 ('not', 0.7125732898712158),
 ('did', 0.6560217142105103),
 ('does', 0.6495506167411804),
 ('try', 0.5491036176681519),
 ('we', 0.5474551320075989),
 ('ask', 0.5471649765968323),
 ('if', 0.545981764793396),
 ('go', 0.5437604784965515),
 ('else', 0.5414758324623108)]